In [2]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

drive.mount('/content/drive') # montamos el drive

# cargamos el dataframe de entrenamiento y el dataframe de testeo
# el dataframe de entrenamiento tiene toda la información, incluyendo la columna 'Severity' (severidad)
# esta es la columna que vamos a entrenar el modelo para predecir: 0=severidad baja; 1=severidad alta.
# en cambio, el dataframe de test no tiene esta columna: son los datos que usamos para generar la predicción que se entrega 
filename_train = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/train.csv' 
filename_test = '/content/drive/My Drive/LaboDatos2022/SegundaEjercitacion/test4alumnxs.csv' 

df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)

Mounted at /content/drive


In [17]:
y = np.array(df_train['Severity']) # armo el vector de etiquetas

#corrijo los strings a numeros

#sin esto==0.781 /// con esto =0.784 -----> no sirven de casi nada
df_train.loc[df_train.Sunrise_Sunset == 'Day', 'Sunrise_Sunset']= 1
df_train.loc[df_train.Sunrise_Sunset == 'Night', 'Sunrise_Sunset']= 0

df_train.loc[df_train.Civil_Twilight == 'Day', 'Civil_Twilight']= 1
df_train.loc[df_train.Civil_Twilight == 'Night', 'Civil_Twilight']= 0

df_train.loc[df_train.Nautical_Twilight == 'Day', 'Nautical_Twilight']= 1
df_train.loc[df_train.Nautical_Twilight == 'Night', 'Nautical_Twilight']= 0

df_train.loc[df_train.Astronomical_Twilight == 'Day', 'Astronomical_Twilight']= 1
df_train.loc[df_train.Astronomical_Twilight == 'Night', 'Astronomical_Twilight']= 0


#sin esto==0.781 /// con esto = 0.777 ---> baja
df_train.loc[df_train.Side == 'R', 'Side']= 1
df_train.loc[df_train.Side == 'L', 'Side']= 0

#sin esto==0.781 /// con esto = 0.780 ---> baja
df_train.loc[df_train.Amenity == 'TRUE', 'Amenity']= 0
df_train.loc[df_train.Amenity == 'FALSE', 'Amenity']= 1


#sin esto==0.781  /// con esto==0.783 ---> vi los dataset y no importan casi nada, son todos false en casi todos
df_train.loc[df_train.Traffic_Signal == 'TRUE', 'Traffic_Signal']= 0
df_train.loc[df_train.Traffic_Signal == 'FALSE', 'Traffic_Signal']= 1

df_train.loc[df_train.Traffic_Calming == 'TRUE', 'Traffic_Calming']= 0
df_train.loc[df_train.Traffic_Calming == 'FALSE', 'Traffic_Calming']= 1

df_train.loc[df_train.Railway == 'TRUE', 'Railway']= 0
df_train.loc[df_train.Railway == 'FALSE', 'Railway']= 1



#estoy viendo la relacion entre los severidad 1 y las condiciones climaticas a ver si es worth meterlas en el modelo
columnasxd = ['Severity', 'Weather_Condition']
df = df_train[columnasxd]



indexNames = df[ df['Severity'] == 0 ].index
df.drop(indexNames , inplace=True)

#print(df)

print(df['Weather_Condition'].value_counts())
print('///////////////////////////////////////////////////////////')
print(df_train['Weather_Condition'].value_counts())



# mi modelo ctm
columnas_clima = ['Distance(mi)','Temperature(F)','Wind_Chill(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(in)']

Fair                           7818
Cloudy                         3125
Mostly Cloudy                  1988
Partly Cloudy                  1479
Light Rain                      972
Light Snow                      531
Fog                             320
Overcast                        206
Rain                            201
Fair / Windy                    180
Smoke                           102
Haze                             90
Snow                             72
Heavy Rain                       72
Light Drizzle                    67
Cloudy / Windy                   55
Mostly Cloudy / Windy            54
Thunder in the Vicinity          52
Wintry Mix                       45
T-Storm                          41
Partly Cloudy / Windy            34
Light Rain with Thunder          32
Light Rain / Windy               32
Thunder                          28
Drizzle                          25
Heavy T-Storm                    19
Light Snow / Windy               18
Heavy Snow                  

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:


# extraigo la matriz de features X
X = np.array(df_train[columnas_clima])




# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

# pruebo con un random forest así como viene de fábrica
clf = RandomForestClassifier(n_estimators=100)  

# ajusto el modelo
clf.fit(X_train, y_train)

# armo el vector de predicciones.
y_hat_test = clf.predict_proba(X_test)[:, 1]
y_hat_train = clf.predict_proba(X_train)[:, 1]

# evaluo el AUC
roc_test = roc_auc_score(y_test, y_hat_test)
roc_train = roc_auc_score(y_train, y_hat_train)
roc_test, roc_train


(0.7833321232508997, 0.9999806336154666)